In [665]:
from PIL import Image
import torch
import torchvision.transforms as transforms

In [666]:
img = Image.open('captures2/2023_05_19_19_02_56.613060 n.png')

In [667]:
img = img.resize((480, 480))

In [668]:
# transform to numpy

In [669]:
img = transforms.PILToTensor()(img).to(torch.float32)

In [670]:
img.shape

torch.Size([3, 480, 480])

In [115]:
import torch
import torch.nn as nn


class ConvolutionBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding):
        super().__init__()
        self.conv_block = nn.Sequential(
            nn.Conv2d(in_channels=in_channels, out_channels=out_channels,
                      kernel_size=kernel_size, stride=stride, padding=padding),
            nn.BatchNorm2d(out_channels),
            nn.ReLU())

    def forward(self, x):
        return self.conv_block(x)


class STEM(nn.Module):

    def __init__(self):
        super(STEM, self).__init__()
        self.conv1 = ConvolutionBlock(
            in_channels=3, out_channels=32, kernel_size=3, stride=2, padding=0)
        self.conv2 = ConvolutionBlock(
            in_channels=32, out_channels=32, kernel_size=3, stride=1, padding=0)
        self.conv3 = ConvolutionBlock(
            in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=0)
        self.conv4 = ConvolutionBlock(
            in_channels=64, out_channels=80, kernel_size=1, stride=1, padding=0)
        self.conv5 = ConvolutionBlock(
            in_channels=80, out_channels=192, kernel_size=3, stride=1, padding=0)
        self.pool2 = nn.MaxPool2d(kernel_size=3, stride=2, padding=0)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)  # 149 x 149 x 64
        x = self.pool1(x)  # 147 x 147 x 64
        x = self.conv4(x)  # 147 x 147 x 80
        x = self.conv5(x)  # 147 x 147 x 192
        x = self.pool2(x)  # 73 x 73 x 192
        return x


class InceptionBlockA(nn.Module):
    def __init__(self, in_channels):
        super(InceptionBlockA, self).__init__()
        
        self.branch1 = nn.Sequential(
            ConvolutionBlock(in_channels=in_channels, out_channels=64,
                             kernel_size=1, stride=1, padding=0),
            ConvolutionBlock(in_channels=64, out_channels=96,
                             kernel_size=3, stride=1, padding=1),
            ConvolutionBlock(in_channels=96, out_channels=96, kernel_size=3, stride=1, padding=1))
        
        self.branch2 = nn.Sequential(
            ConvolutionBlock(in_channels=in_channels, out_channels=48,
                             kernel_size=1, stride=1, padding=0),
            ConvolutionBlock(in_channels=48, out_channels=64, kernel_size=3, stride=1, padding=1))
        
        self.branch3 = nn.Sequential(
            nn.AvgPool2d(kernel_size=3, stride=1, padding=1),
            ConvolutionBlock(in_channels=in_channels, out_channels=64, kernel_size=1, stride=1, padding=0))
        
        self.branch4 = ConvolutionBlock(
            in_channels=in_channels, out_channels=64, kernel_size=1, stride=1, padding=0)

    def forward(self, x):
        return torch.cat([self.branch1(x), self.branch2(x), self.branch3(x), self.branch4(x)], dim=1)


class ReductionBlockA(nn.Module):
    def __init__(self, in_channels):
        super(ReductionBlockA, self).__init__()

        self.branch1 = nn.Sequential(
            ConvolutionBlock(in_channels=in_channels, out_channels=64,
                             kernel_size=1, stride=1, padding=0),
            ConvolutionBlock(in_channels=64, out_channels=96,
                             kernel_size=3, stride=1, padding=1),
            ConvolutionBlock(in_channels=96, out_channels=96, kernel_size=3, stride=2, padding=0))
        
        self.branch2 = nn.Sequential(
            ConvolutionBlock(in_channels=in_channels, out_channels=384, kernel_size=2, stride=2, padding=0))
        
        self.branch3 = nn.MaxPool2d(kernel_size=3, stride=2, padding=0)

    def forward(self, x):
        return torch.cat([self.branch1(x), self.branch2(x), self.branch3(x)], dim=1)


class InceptionBlockB(nn.Module):
    def __init__(self, in_channels, nbr_kernel):
        super(InceptionBlockB, self).__init__()

        self.branch1 = nn.Sequential(
            ConvolutionBlock(in_channels=in_channels, out_channels=nbr_kernel,
                             kernel_size=1, stride=1, padding=0),
            ConvolutionBlock(in_channels=nbr_kernel, out_channels=nbr_kernel, kernel_size=(
                7, 1), stride=1, padding=(0,3)),
            ConvolutionBlock(in_channels=nbr_kernel, out_channels=nbr_kernel, kernel_size=(
                1, 7), stride=1, padding=(3,0)),
            ConvolutionBlock(in_channels=nbr_kernel, out_channels=nbr_kernel, kernel_size=(
                7, 1), stride=1, padding=(0, 3)),
            ConvolutionBlock(in_channels=nbr_kernel, out_channels=192, kernel_size=(1, 7), stride=1, padding=(3,0)))

        self.branch2 = nn.Sequential(
            ConvolutionBlock(in_channels=in_channels, out_channels=nbr_kernel,
                             kernel_size=1, stride=1, padding=0),
            ConvolutionBlock(in_channels=nbr_kernel, out_channels=nbr_kernel, kernel_size=(
                1, 7), stride=1, padding=(0,3)),
            ConvolutionBlock(in_channels=nbr_kernel, out_channels=192, kernel_size=(7, 1), stride=1, padding=(3,0)))
        
        self.branch3 = nn.Sequential(
            nn.AvgPool2d(kernel_size=3, stride=1, padding=1),
            ConvolutionBlock(in_channels=in_channels, out_channels=192, kernel_size=1, stride=1, padding=0))
        
        self.branch4 = ConvolutionBlock(
            in_channels=in_channels, out_channels=192, kernel_size=1, stride=1, padding=0)

    def forward(self, x):
        return torch.cat([self.branch1(x), self.branch2(x), self.branch3(x), self.branch4(x)], dim=1)


class ReductionBlockB(nn.Module):
    def __init__(self, in_channels):
        super(ReductionBlockB, self).__init__()

        self.branch1 = nn.Sequential(
            ConvolutionBlock(in_channels=in_channels, out_channels=192,
                             kernel_size=1, stride=1, padding=0),
            ConvolutionBlock(in_channels=192, out_channels=192,
                             kernel_size=(1, 7), stride=1, padding=(0, 3)),
            ConvolutionBlock(in_channels=192, out_channels=192,
                             kernel_size=(7, 1), stride=1, padding=(3,0)),
            ConvolutionBlock(in_channels=192, out_channels=192, kernel_size=3, stride=2, padding=0))
        
        self.branch2 = nn.Sequential(
            ConvolutionBlock(in_channels=in_channels, out_channels=192,
                             kernel_size=1, stride=1, padding=0),
            ConvolutionBlock(in_channels=192, out_channels=320, kernel_size=3, stride=2, padding=0))
        
        self.branch3 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=2))

    def forward(self, x):
        return torch.cat([self.branch1(x), self.branch2(x), self.branch3(x)], dim=1)


class AuxiliaryClassifierBlock(nn.Module):
    def __init__(self, in_channels):
        super(AuxiliaryClassifierBlock, self).__init__()
        self.branch1 = nn.Sequential(
            nn.AvgPool2d(kernel_size=5, stride=3, padding=0),
            ConvolutionBlock(in_channels=in_channels, out_channels=128,
                             kernel_size=1, stride=1, padding=0),
            ConvolutionBlock(in_channels=128, out_channels=768,
                                 kernel_size=1, stride=1, padding=0),
            )

        self.fc1 = nn.LazyLinear(1024)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(1024, 1000)

    
    def forward(self, x):
        x = self.branch1(x)
        x = torch.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return nn.Softmax()(x)


class InceptionBlockC(nn.Module):
    def __init__(self, in_channels):
        super().__init__()

        self.branch1 = nn.Sequential(
            ConvolutionBlock(in_channels=in_channels, out_channels=384, kernel_size=1, stride=1, padding=0))
        
        self.branch1_1 = nn.Sequential(
            ConvolutionBlock(in_channels=384, out_channels=384, kernel_size=(1, 3), stride=1, padding=(0,1)))
        
        self.branch1_2 = nn.Sequential(
            ConvolutionBlock(in_channels=384, out_channels=384,
                             kernel_size=(3,1), stride=1, padding=(1,0))
        )
        self.branch2 = nn.Sequential(
            ConvolutionBlock(in_channels=in_channels, out_channels=448,
                             kernel_size=1, stride=1, padding=0),
            ConvolutionBlock(in_channels=448, out_channels=384,kernel_size= 3, stride = 1 ,padding=1)
        )
        self.branch2_1 = ConvolutionBlock(
            in_channels=384, out_channels=384, kernel_size=(1, 3), stride=1, padding=(0,1))
        
        self.branch2_2 = ConvolutionBlock(
            in_channels=384, out_channels=384, kernel_size=(3, 1), stride=1, padding=(1,0))
        
        self.branch3 = nn.Sequential(
            nn.AvgPool2d(kernel_size=3, stride=1, padding=1),
            ConvolutionBlock(in_channels=in_channels, out_channels=192, kernel_size=1, stride=1, padding=0)
        )

        self.branch4 = ConvolutionBlock(
            in_channels=in_channels, out_channels=320, kernel_size=1, stride=1, padding=0)

    def forward(self, x):
        return torch.cat([self.branch1_1(self.branch1(x)), self.branch1_2(self.branch1(x)), self.branch2_1(self.branch2(x)), self.branch2_2(self.branch2(x)), self.branch3(x), self.branch4(x)], dim=1)


class InceptionV3(nn.Module):
    def __init__(self):
        super().__init__()

        self.stem = STEM()

        self.InceptionA1 = InceptionBlockA(192)
        self.InceptionA2 = InceptionBlockA(288)
        self.InceptionA3 = InceptionBlockA(288)

        self.redA = ReductionBlockA(288)

        self.InceptionB1 = InceptionBlockB(768, 128)
        self.InceptionB2 = InceptionBlockB(768,160)
        self.InceptionB3 = InceptionBlockB(768,160)
        self.InceptionB4 = InceptionBlockB(768,192)

        self.redB = ReductionBlockB(768)


        self.InceptionC1 = InceptionBlockC(1280)
        self.InceptionC2 = InceptionBlockC(2048)

        self.aux = AuxiliaryClassifierBlock(768)

        self.pool = nn.AvgPool2d(kernel_size = 1)
        self.fc1 = nn.LazyLinear( 1000)

        self.fc2 = nn.Linear(1000, 2)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):

        x = self.stem(x)

        x = self.InceptionA1(x)
        x = self.InceptionA2(x)
        x = self.InceptionA2(x)

        x = self.redA(x)

        x = self.InceptionB1(x)
        x = self.InceptionB2(x)
        x = self.InceptionB3(x)
        x = self.InceptionB4(x)

        aux = self.aux(x)

        x = self.redB(x)

        x = self.InceptionC1(x)
        x = self.InceptionC2(x)

        x = self.pool(x)
        x = torch.flatten(x,1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.softmax(x)


        return x


In [116]:
import pandas as pd
from dataset import DinoDataset
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.transforms import CenterCrop, Resize, Compose, ToTensor, Normalize
from tqdm import tqdm
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import gc



transformer = Compose([
    Resize((480, 480)),
    CenterCrop(480),
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [117]:
key_frame = pd.read_csv('labels_dino.csv')
train, test = train_test_split(key_frame, test_size=0.2)

train = pd.DataFrame(train)
test = pd.DataFrame(test)

batch_size = 4

trainset = DinoDataset(root_dir="captures2",
                       dataframe=train, transform=transformer)
trainloader = DataLoader(trainset, batch_size=batch_size)

testset = DinoDataset(root_dir="captures2",
                      dataframe=test, transform=transformer)
testloader = DataLoader(testset, batch_size=batch_size)

device = 'cpu'

In [118]:
model = InceptionV3()

model = model.to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.009)


def train_model(model, criterion, optimizer, num_epochs=25):
    epochs = 15  # number of training passes over the mini batches
    loss_container = []  # container to store the loss values after each epoch
    for epoch in range(epochs):  # loop over the dataset multiple times

        running_loss = 0.0
        for data in tqdm(trainloader, position=0, leave=True):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        loss_container.append(running_loss)

        print(f'[{epoch + 1}] | loss: {running_loss / len(trainloader):.3f}')
        running_loss = 0.0

    print('Finished Training')
    torch.save(model.state_dict(), 'path_to_save_model.pth')

    # plot the loss curve
    plt.plot(np.linspace(1, epochs, epochs).astype(int), loss_container)
    plt.show()

    # clean up the gpu memory
    gc.collect()
    torch.cuda.empty_cache()

/opt/homebrew/lib/python3.10/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [119]:
train_model(model, criterion, optimizer, num_epochs=25)

  0%|          | 0/1043 [00:00<?, ?it/s]/var/folders/sl/6pfddh5x4ljcbvrtvkcnqglc0000gn/T/ipykernel_1623/2110645663.py:173: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return nn.Softmax()(x)
 56%|█████▌    | 583/1043 [28:47<22:43,  2.96s/it]


KeyboardInterrupt: 

In [98]:
for x, y in trainloader:
    a = x
    break

In [101]:
inc = InceptionV3().to('cpu')

In [102]:
inc(a.to('cpu'))

/var/folders/sl/6pfddh5x4ljcbvrtvkcnqglc0000gn/T/ipykernel_1623/3699613788.py:173: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return nn.Softmax()(x)
/var/folders/sl/6pfddh5x4ljcbvrtvkcnqglc0000gn/T/ipykernel_1623/3699613788.py:272: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)


tensor([[0.4536, 0.5464],
        [0.5021, 0.4979],
        [0.5219, 0.4781],
        [0.5793, 0.4207]], grad_fn=<SoftmaxBackward0>)